In [3]:
import pandas as pd
import numpy as np
import math

In [268]:
df = pd.read_csv('data/XBTUSDC_60.csv', header=None,
                 names=['time', 'open', 'high', 'low', 'close', 'volume', 'trades'])
df.set_index('time', inplace=True)
df.index = pd.DatetimeIndex(df.index * 1e+9)

In [269]:
len(df)

9215

In [5]:
df.head()

,open,high,low,close,volume,trades
time,,,,,,
2020-01-08 15:00:00,8300.00,8300.00,8300.00,8300.00,0.002410,2
2020-01-08 16:00:00,8217.51,8217.51,8217.51,8217.51,0.012290,2
2020-01-08 17:00:00,7900.01,7949.99,7870.00,7949.99,0.052487,10
2020-01-08 18:00:00,7949.99,8000.00,7870.99,7870.99,0.038592,8
2020-01-08 20:00:00,7870.98,7999.99,7870.00,7999.99,0.044715,5


In [6]:
type(math.sqrt(4))

float

In [7]:
np.std([3,4],ddof=0), math.sqrt((0.5**2 + 0.5**2)/1)

(0.5, 0.7071067811865476)

In [8]:
np.var([3,4]), np.std([3,4])**2

(0.25, 0.25)

In [246]:
class Trader1:
    # Bollinger: https://www.investopedia.com/trading/using-bollinger-bands-to-gauge-trends/#:~:text=Using%20the%20bands%20as%20overbought,have%20deviated%20from%20the%20mean.
    def __init__(self, n_cycles, n_devs, qty):
        # qty = quantity to trade at a time
        # EMA = Closing price x multiplier + EMA (previous day) x (1-multiplier)
        # 
        # BOLU=MA(TP,n)+m∗σ[TP,n]
        # BOLD=MA(TP,n)−m∗σ[TP,n]
        # where:
        # BOLU=Upper Bollinger Band
        # BOLD=Lower Bollinger Band
        # MA=Moving average
        # TP (typical price)=(High+Low+Close)÷3
        # n=Number of cycles in smoothing period ... n_cycles
        # m=Number of standard deviations ... n_devs
        # σ[TP,n]=Standard Deviation over last n periods of TP
        self.N = n_cycles
        self.M = n_devs
        self.qty = qty
        self.prices = np.array([0.] * self.N)
        
        self.sma = None
        self.upper = None
        self.lower = None
        self.var = None
        self.std = None
    
        self.position = 0
        self.trades = []
        
        self.warming_up = True
        self.i = 0
    
    def update(self, cycle, ohlc: dict, price):
        _, high, low, close = self._unpack(ohlc)

        last_price = self.prices[self.i]
        self.prices[self.i] = price
        self.i += 1
        if self.i == len(self.prices):
            self.i = 0
            if self.warming_up:
                self.warming_up = False
                self.sma = np.mean(self.prices)
                self.var = np.var(self.prices)
                self.std = math.sqrt(self.var)
                self.upper = self.sma + self.std
                self.lower = self.sma - self.std
                return

        if not self.warming_up:
            old_sma = self.sma
            self.sma += (price - last_price) / self.N
            self.var += (price - self.sma + last_price - old_sma)*(price - last_price)/(self.N)
            self.var = self.check_var(self.var)
            self.std = math.sqrt(self.var)
            self.upper = self.sma + self.std
            self.lower = self.sma - self.std

    def check(self, x):
        if np.abs(x) > 0.01:
            raise
    
    def check_var(self, var):
        if var < 0:
            if abs(var) >= 1.e-6:
                raise Exception(f'var was {var}')
            return 0.
        return var
    
    def trade(self) -> float:  # return number of shares to trade
        if self.warming_up:
            return None
        # buy
        if self.prices[self.i] < self.lower and self.position <= 0:
            self.position += self.qty
            return self.qty
    
        # sell
        if self.prices[self.i] > self.upper and self.position >= 0:
            self.position -= self.qty
            return -self.qty
        
    
    @staticmethod
    def _unpack(ohlc: dict):
        return ohlc['open'], ohlc['high'], ohlc['low'], ohlc['close']

In [235]:
def trade(df, model):
    # model needs to have .update(cycle, ohlc) and trade()
    # return the trades

    trades = pd.DataFrame(index=df.index)
    trades['qty'] = [0.] * len(trades)
    
    for i, (cycle, ohlc) in enumerate(df.iterrows()):
        model.update(cycle, ohlc, ohlc['close'])
        qty = model.trade()
        if qty:
            trades.iloc[i] = qty
    return trades

def account(df, trades, fee=0.0026):
    prices = df.open
    volume = prices.shift(-1, fill_value=prices.iloc[-1]) * trades.qty * (1 + np.sign(trades.qty) * fee)

    last_price = prices.iloc[-1]
    qty_to_liquidate = -trades.qty.sum()
    last_volume = last_price * qty_to_liquidate * (1 + np.sign(qty_to_liquidate) * fee)

    pnl = -(last_volume + volume.sum())
    
    return pnl

def cycle_realized_pnl(df, trades, fee=0.0026):
    prices = df.open
    volume = prices.shift(-1, fill_value=prices.iloc[-1]) * trades.qty * (1 + np.sign(trades.qty) * fee)
    
    values = pd.Series(index=prices.index, dtype=np.float64).fillna(0.)
    v = 0.
    position = 0.
    for i, qty in enumerate(trades.qty):
        was_open = position != 0
        
        position = round(position + qty, 8)
        v -= volume.iloc[i]
        
        if position == 0:
            if was_open:
                values[i] = v
            v = 0
            
    if position:
        last_price = prices.iloc[-1]
        assert position == trades.qty.sum()
        qty_to_liquidate = -position
        last_volume = last_price * qty_to_liquidate * (1 + np.sign(qty_to_liquidate) * fee)
        assert values.iloc[-1] == 0
        values.iloc[-1] = -last_volume + v
    return values


def sharpe(pnls):
    return np.sum(pnls) / np.std(pnls)

def test_account():
    df = pd.DataFrame({'open': [10,11,12,13,14]})
    trades = pd.DataFrame({'qty': [1,-1,1,0,0]})
    fee = 1
    a = cycle_realized_pnl(df, trades, fee=fee).sum()
    b = account(df, trades, fee=fee)
    assert a == b
    assert a == -48.    

test_account()

In [236]:
trades = trade(df, Trader1(10, 2, 0.002))
sharpe(cycle_realized_pnl(df, trades))

In [238]:
sharpe(cycle_realized_pnl(df, trades))

-260.23484555398574

# Plotting

In [1]:
import plotly.graph_objs as go
import plotly.offline as offline

offline.init_notebook_mode(connected=True)

In [ ]:
trace = go.Scatter(
    x=trades.index,
    y=trades['price'],
    name='Tick'
    # mode='markers',
    # marker=dict(
    #     size=10,
    #     color='rgba(182, 255, 193, .9)',
    #     line=dict(
    #         width=2,
    #     )
    # )
)

# Analysing

sharpe = sum(return) / std(return)

Trader1: n_cycles, n_devs, qty

In [252]:
results = {}

In [253]:
model = Trader1
for n_cycles in range(80,200,2):
    for n_devs in [1]:
        for qty in [0.002]:
            print(f'Running model({n_cycles}, {n_devs}, {qty})', end='...')
            trades = trade(df, model(n_cycles, n_devs, qty))
            n_trades = len(trades[trades.qty != 0])
            s = sharpe(cycle_realized_pnl(df, trades))
            results[(n_cycles, n_devs, qty)] = (s, n_trades)
            print(f' Got sharpe = {s}, n_trades = {n_trades}')

Running model(80, 1, 0.002)... Got sharpe = -62.927979033923584, n_trades = 249
Running model(82, 1, 0.002)... Got sharpe = -38.48547976421766, n_trades = 243
Running model(84, 1, 0.002)... Got sharpe = -48.76622022102478, n_trades = 239
Running model(86, 1, 0.002)... Got sharpe = -47.35417096943306, n_trades = 231
Running model(88, 1, 0.002)... Got sharpe = -16.028491967711055, n_trades = 229
Running model(90, 1, 0.002)... Got sharpe = -9.388969578716706, n_trades = 225
Running model(92, 1, 0.002)... Got sharpe = 18.873831415136898, n_trades = 219
Running model(94, 1, 0.002)... Got sharpe = 8.479512145683174, n_trades = 211
Running model(96, 1, 0.002)... Got sharpe = 28.955295825309037, n_trades = 207
Running model(98, 1, 0.002)... Got sharpe = 35.47326420334236, n_trades = 205
Running model(100, 1, 0.002)... Got sharpe = 63.597410737361066, n_trades = 197
Running model(102, 1, 0.002)... Got sharpe = 87.49540920299229, n_trades = 197
Running model(104, 1, 0.002)... Got sharpe = 106.36

In [263]:
sorted(results.items(), key=lambda x: results[x[0]][0])

[((80, 1, 0.002), (-62.927979033923584, 249)),
 ((84, 1, 0.002), (-48.76622022102478, 239)),
 ((86, 1, 0.002), (-47.35417096943306, 231)),
 ((82, 1, 0.002), (-38.48547976421766, 243)),
 ((88, 1, 0.002), (-16.028491967711055, 229)),
 ((90, 1, 0.002), (-9.388969578716706, 225)),
 ((94, 1, 0.002), (8.479512145683174, 211)),
 ((92, 1, 0.002), (18.873831415136898, 219)),
 ((186, 1, 0.002), (28.455407430237514, 111)),
 ((96, 1, 0.002), (28.955295825309037, 207)),
 ((98, 1, 0.002), (35.47326420334236, 205)),
 ((118, 1, 0.002), (35.58322296870231, 177)),
 ((116, 1, 0.002), (37.870835750335296, 179)),
 ((184, 1, 0.002), (39.18734788644151, 117)),
 ((140, 1, 0.002), (46.428284985128755, 153)),
 ((126, 1, 0.002), (50.58449530836365, 167)),
 ((188, 1, 0.002), (55.734573624040756, 112)),
 ((108, 1, 0.002), (57.99444489163017, 183)),
 ((124, 1, 0.002), (59.53448664904926, 169)),
 ((190, 1, 0.002), (63.001618708325495, 110)),
 ((100, 1, 0.002), (63.597410737361066, 197)),
 ((130, 1, 0.002), (64.03617

In [264]:
160 / 24


6.666666666666667

In [265]:
df

,open,high,low,close,volume,trades
time,,,,,,
2020-01-08 15:00:00,8300.00,8300.00,8300.00,8300.00,0.002410,2
2020-01-08 16:00:00,8217.51,8217.51,8217.51,8217.51,0.012290,2
2020-01-08 17:00:00,7900.01,7949.99,7870.00,7949.99,0.052487,10
2020-01-08 18:00:00,7949.99,8000.00,7870.99,7870.99,0.038592,8
2020-01-08 20:00:00,7870.98,7999.99,7870.00,7999.99,0.044715,5
...,...,...,...,...,...,...
2021-03-31 19:00:00,59420.00,59534.68,58604.83,58604.83,16.042371,171
2021-03-31 20:00:00,58644.28,59114.19,58421.40,59009.58,1.642617,121
2021-03-31 21:00:00,59057.49,59158.25,58787.89,58787.89,0.907528,52
